# CHƯƠNG 8 Xử lý dữ liệu: Nối, Kết hợp và Định hình lại

## 8.1 Lập chỉ mục phân cấp

### Lập chỉ mục phân cấp là một tính năng quan trọng của pandas, cho phép bạn có nhiều (hai hoặc nhiều) mức chỉ mục trên một trục. Một cách khác để hiểu về nó là nó cung cấp cho bạn cách làm việc với dữ liệu có chiều cao hơn ở dạng chiều thấp hơn. Hãy bắt đầu với một ví dụ đơn giản: tạo một Series với danh sách các danh sách (hoặc mảng) làm chỉ mục:

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

### Những gì bạn đang thấy là một cái nhìn đẹp mắt về một Chuỗi với Đa Chỉ số làm chỉ số của nó. Các "khoảng trống" trong màn hình chỉ số có nghĩa là "sử dụng nhãn ngay phía trên":

In [3]:
data.index

In [4]:
data["b"]
data["b":"c"]
data.loc[["b", "d"]]

In [5]:
data.loc[:, 2]

### Lập chỉ mục phân cấp đóng vai trò quan trọng trong việc định hình lại dữ liệu và trong các hoạt động dựa trên nhóm như tạo bảng trục. Ví dụ: bạn có thể sắp xếp lại dữ liệu này thành một DataFrame bằng phương thức unstack của nó:

In [6]:
data.unstack()

In [7]:
data.unstack().stack()

In [8]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

In [9]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

### Hãy lưu ý rằng tên chỉ mục "state" và "color" không phải là một phần của nhãn hàng (giá trị frame.index).

In [10]:
frame.index.nlevels

In [11]:
frame["Ohio"]

## Sắp xếp lại và Sắp xếp các Cấp độ
### Đôi khi bạn có thể cần sắp xếp lại thứ tự các cấp độ trên một trục hoặc sắp xếp dữ liệu theo các giá trị trong một cấp độ cụ thể. Phương thức swaplevel lấy hai số hoặc tên cấp độ và trả về một đối tượng mới với các cấp độ được hoán đổi (nhưng dữ liệu không bị thay đổi):

In [12]:
frame.swaplevel("key1", "key2")

### sort_index by default sorts the data lexicographically using all the index levels, but you can choose to use only a single level or a subset of levels to sort by passing the level argument. For example:

In [13]:
frame.sort_index(level=1)
frame.swaplevel(0, 1).sort_index(level=0)

### Hiệu suất lựa chọn dữ liệu sẽ tốt hơn nhiều trên các đối tượng được lập chỉ mục theo thứ bậc nếu chỉ mục được sắp xếp theo thứ tự từ điển bắt đầu từ cấp ngoài cùng—tức là kết quả của việc gọi sort_index(level=0) hoặc sort_index().

## Thống kê Tóm tắt theo Cấp độ
### Nhiều thống kê mô tả và tóm tắt trên DataFrame và Series có tùy chọn cấp độ trong đó bạn có thể chỉ định cấp độ bạn muốn tổng hợp trên một trục ​​cụ thể. Hãy xem xét DataFrame ở trên; chúng ta có thể tổng hợp theo cấp độ trên cả hàng và cột, như sau:

In [14]:
frame.groupby(level="key2").sum()
frame.groupby(level="color", axis="columns").sum()

## Lập chỉ mục với các cột của DataFrame
### Việc muốn sử dụng một hoặc nhiều cột từ DataFrame làm chỉ mục hàng không phải là điều bất thường; ngoài ra, bạn có thể muốn di chuyển chỉ mục hàng vào các cột của DataFrame. Dưới đây là một ví dụ về DataFrame:

In [15]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

In [16]:
frame2 = frame.set_index(["c", "d"])
frame2

In [17]:
frame.set_index(["c", "d"], drop=False)

In [18]:
frame2.reset_index()

## 8.2 Kết hợp và Hợp nhất các Tập dữ liệu
### Dữ liệu chứa trong các đối tượng pandas có thể được kết hợp theo một số cách:
### pandas.merge
### Kết nối các hàng trong DataFrame dựa trên một hoặc nhiều khóa. Điều này sẽ quen thuộc với người dùng SQL hoặc các cơ sở dữ liệu quan hệ khác, vì nó triển khai các thao tác kết nối cơ sở dữ liệu.
### pandas.concat
### Ghép hoặc "xếp chồng" các đối tượng lại với nhau dọc theo một trục.
### combine_first
### Ghép các dữ liệu chồng chéo với nhau để lấp đầy các giá trị bị thiếu trong một đối tượng bằng các giá trị từ một đối tượng khác.

## Nối DataFrame theo kiểu cơ sở dữ liệu
### Các thao tác hợp nhất hoặc nối kết hợp các tập dữ liệu bằng cách liên kết các hàng bằng một hoặc nhiều khóa. Các thao tác này đặc biệt quan trọng trong cơ sở dữ liệu quan hệ (ví dụ: dựa trên SQL).
### Hàm pandas.merge trong pandas là điểm khởi đầu chính để sử dụng các thuật toán này trên dữ liệu của bạn. 
### Hãy bắt đầu với một ví dụ đơn giản:

In [19]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df1
df2

### Đây là một ví dụ về phép nối nhiều-một; dữ liệu trong df1 có nhiều hàng được gắn nhãn a và b, trong khi df2 chỉ có một hàng cho mỗi giá trị trong cột khóa. Gọi pandas.merge với các đối tượng này, chúng ta thu được:

In [20]:
pd.merge(df1, df2)

### Lưu ý rằng tôi không chỉ định cột nào sẽ được nối. Nếu thông tin đó không được chỉ định, pandas.merge sẽ sử dụng các tên cột chồng chéo làm khóa. Tuy nhiên, tốt hơn hết là nên chỉ định rõ ràng:

In [21]:
pd.merge(df1, df2, on="key")

In [22]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

### Bạn có thể nhận thấy các giá trị "c" và "d" cùng dữ liệu liên quan bị thiếu trong kết quả. Theo mặc định, pandas.merge thực hiện phép nối "bên trong"; các khóa trong kết quả là giao điểm, hoặc tập hợp chung được tìm thấy trong cả hai bảng. Các tùy chọn khả thi khác là "trái", "phải" và "ngoài". Phép nối ngoài lấy hợp của các khóa, kết hợp hiệu ứng của việc áp dụng cả phép nối trái và phải:

In [23]:
pd.merge(df1, df2, how="outer")
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

In [24]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df1
df2
pd.merge(df1, df2, on="key", how="left")

In [25]:
pd.merge(df1, df2, how="inner")

In [26]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
pd.merge(left, right, on=["key1", "key2"], how="outer")

### Khi bạn nối các cột với nhau, các chỉ mục trên các đối tượng DataFrame đã truyền sẽ bị loại bỏ. Nếu bạn cần giữ nguyên các giá trị chỉ mục, bạn có thể sử dụng reset_index để thêm chỉ mục vào các cột.

In [27]:
pd.merge(left, right, on="key1")

In [28]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

### Hợp nhất trên Chỉ mục
### Trong một số trường hợp, khóa hợp nhất trong DataFrame sẽ được tìm thấy trong chỉ mục (nhãn hàng) của nó. Trong trường hợp này, bạn có thể truyền left_index=True hoặc right_index=True (hoặc cả hai) để chỉ định rằng chỉ mục nên được sử dụng làm khóa hợp nhất:

In [29]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
left1
right1
pd.merge(left1, right1, left_on="key", right_index=True)

### Nếu bạn xem xét kỹ ở đây, bạn sẽ thấy các giá trị chỉ mục cho left1 đã được giữ nguyên, trong khi ở các ví dụ khác ở trên, các chỉ mục của các đối tượng DataFrame đầu vào bị loại bỏ. Vì chỉ mục của right1 là duy nhất, phép hợp nhất "nhiều-một" này (với phương thức how="inner" mặc định) có thể giữ nguyên các giá trị chỉ mục từ left1 tương ứng với các hàng trong đầu ra.

In [30]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

In [31]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
lefth
righth

In [32]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

In [33]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
left2
right2
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

In [34]:
left2.join(right2, how="outer")

In [35]:
left1.join(right1, on="key")

In [36]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another
left2.join([right2, another])
left2.join([right2, another], how="outer")

### Ghép nối dọc theo trục
### Một loại thao tác kết hợp dữ liệu khác được gọi thay thế cho nhau là ghép nối hoặc xếp chồng. Hàm ghép nối của NumPy có thể thực hiện điều này với các mảng NumPy:

In [37]:
arr = np.arange(12).reshape((3, 4))
arr
np.concatenate([arr, arr], axis=1)

## 8.2 Kết hợp và hợp nhất các tập dữ liệu

## Tham gia DataFrame theo kiểu cơ sở dữ liệu

In [38]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [39]:
s1
s2
s3
pd.concat([s1, s2, s3])

In [40]:
pd.concat([s1, s2, s3], axis="columns")

In [41]:
s4 = pd.concat([s1, s3])
s4
pd.concat([s1, s4], axis="columns")
pd.concat([s1, s4], axis="columns", join="inner")

In [42]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result
result.unstack()

In [43]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

In [44]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df1
df2
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

In [45]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

In [46]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"])

In [47]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df1
df2

In [48]:
pd.concat([df1, df2], ignore_index=True)

### Kết hợp Dữ liệu với Chồng chéo
### Có một tình huống kết hợp dữ liệu khác không thể được biểu diễn bằng phép hợp nhất hoặc phép nối. Bạn có thể có hai tập dữ liệu với các chỉ mục chồng chéo toàn bộ hoặc một phần. Để minh họa, hãy xem xét hàm where của NumPy, thực hiện tương đương với biểu thức if-else hướng mảng:

In [49]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
a
b
np.where(pd.isna(a), b, a)

In [50]:
a.combine_first(b)

In [51]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

## 8.3 Định hình lại và Xoay trục
### Có một số thao tác cơ bản để sắp xếp lại dữ liệu dạng bảng. Chúng được gọi là thao tác định hình lại hoặc xoay trục.
### Định hình lại bằng Lập chỉ mục Phân cấp
### Lập chỉ mục phân cấp cung cấp một cách nhất quán để sắp xếp lại dữ liệu trong một DataFrame.
### Có hai thao tác chính:
### xếp chồng
### Thao tác này "xoay" hoặc xoay trục từ các cột trong dữ liệu sang các hàng.
### hủy xếp chồng
### Thao tác này xoay trục từ các hàng sang các cột.
### Tôi sẽ minh họa các thao tác này thông qua một loạt ví dụ. Hãy xem xét một Data-Frame nhỏ với các mảng chuỗi làm chỉ mục hàng và cột:

In [52]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

In [53]:
result = data.stack()
result

In [54]:
result.unstack()

In [55]:
result.unstack(level=0)
result.unstack(level="state")

In [56]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

In [57]:
data2.unstack()
data2.unstack().stack()
data2.unstack().stack(dropna=False)

In [58]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df
df.unstack(level="state")

In [59]:
df.unstack(level="state").stack(level="side")

### Chuyển đổi định dạng "Dài" sang "Rộng"
### Một cách phổ biến để lưu trữ nhiều chuỗi thời gian trong cơ sở dữ liệu và tệp CSV là đôi khi được gọi là định dạng dài hoặc định dạng xếp chồng. Ở định dạng này, các giá trị riêng lẻ được biểu diễn bằng một hàng duy nhất trong bảng thay vì nhiều giá trị trên mỗi hàng.

In [60]:
data = pd.read_csv("examples/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

In [61]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods
data.index = periods.to_timestamp("D")
data.head()

In [62]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

In [63]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

In [64]:
long_data[:10]

In [65]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

In [66]:
long_data.index.name = None

In [67]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

In [68]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()
pivoted["value"].head()

In [69]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

### Xoay định dạng "Rộng" sang "Dài"
### Một phép toán ngược để xoay cho DataFrame là pandas.melt. Thay vì biến đổi một cột thành nhiều cột trong một DataFrame mới, nó sẽ hợp nhất nhiều cột thành một, tạo ra một DataFrame dài hơn dữ liệu đầu vào. Hãy xem một ví dụ:

In [71]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

In [72]:
melted = pd.melt(df, id_vars="key")
melted

In [73]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

In [74]:
reshaped.reset_index()

In [75]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

In [76]:
pd.melt(df, value_vars=["A", "B", "C"])
pd.melt(df, value_vars=["key", "A", "B"])

## 8.4 Kết luận
### Giờ đây, khi bạn đã nắm được một số kiến ​​thức cơ bản về Pandas để nhập, làm sạch và sắp xếp dữ liệu, chúng ta đã sẵn sàng chuyển sang trực quan hóa dữ liệu với matplotlib. Chúng ta sẽ quay lại khám phá các khía cạnh khác của Pandas ở phần sau của cuốn sách khi thảo luận về các phân tích nâng cao hơn.